https://github.com/keras-team/keras/blob/master/examples/cifar10_cnn.py

In [1]:
'''Train a simple deep CNN on the CIFAR10 small images dataset.
It gets to 75% validation accuracy in 25 epochs, and 79% after 50 epochs.
(it's still underfitting at that point, though).


'''

from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os


Using TensorFlow backend.


In [5]:
batch_size = 32
num_classes = 10
epochs = 10
data_augmentation = True
num_predictions = 20
save_dir = "drive/Colab Notebooks/GAN Medical Image/Models"
model_name = 'keras_cifar10_trained_model.h5'

# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [3]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 64)        18496     
__________

In [0]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [10]:
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)



Using real-time data augmentation.
Epoch 1/10
1563/1563 [==============================] - 43s 28ms/step - loss: 1.1396 - acc: 0.6118 - val_loss: 1.1962 - val_acc: 0.6154
Epoch 2/10
 115/1563 [=>............................] - ETA: 37s - loss: 1.1271 - acc: 0.6149

1563/1563 [==============================] - 44s 28ms/step - loss: 1.1287 - acc: 0.6171 - val_loss: 1.1069 - val_acc: 0.6410
Epoch 3/10
 509/1563 [========>.....................] - ETA: 28s - loss: 1.1218 - acc: 0.6203

1563/1563 [==============================] - 44s 28ms/step - loss: 1.1233 - acc: 0.6204 - val_loss: 1.2401 - val_acc: 0.6178
Epoch 4/10
 663/1563 [===========>..................] - ETA: 24s - loss: 1.1251 - acc: 0.6226

1563/1563 [==============================] - 44s 28ms/step - loss: 1.1225 - acc: 0.6233 - val_loss: 1.1307 - val_acc: 0.6312
Epoch 5/10
 699/1563 [============>.................] - ETA: 23s - loss: 1.1243 - acc: 0.6181

1563/1563 [==============================] - 44s 28ms/step - loss: 1.1179 - acc: 0.6217 - val_loss: 0.9918 - val_acc: 0.6706
Epoch 6/10
 712/1563 [============>.................] - ETA: 23s - loss: 1.1171 - acc: 0.6261

1563/1563 [==============================] - 44s 28ms/step - loss: 1.1203 - acc: 0.6241 - val_loss: 0.9511 - val_acc: 0.6832
Epoch 7/10
 713/1563 [============>.................] - ETA: 22s - loss: 1.1073 - acc: 0.6238

1563/1563 [==============================] - 44s 28ms/step - loss: 1.1146 - acc: 0.6240 - val_loss: 1.0219 - val_acc: 0.6620
Epoch 8/10
 730/1563 [=============>................] - ETA: 22s - loss: 1.1043 - acc: 0.6270

1563/1563 [==============================] - 44s 28ms/step - loss: 1.1108 - acc: 0.6248 - val_loss: 1.1043 - val_acc: 0.6438
Epoch 9/10
 740/1563 [=============>................] - ETA: 22s - loss: 1.1127 - acc: 0.6282

1563/1563 [==============================] - 44s 28ms/step - loss: 1.1151 - acc: 0.6241 - val_loss: 1.1331 - val_acc: 0.6239
Epoch 10/10
 732/1563 [=============>................] - ETA: 22s - loss: 1.1205 - acc: 0.6259

1563/1563 [==============================] - 44s 28ms/step - loss: 1.1143 - acc: 0.6282 - val_loss: 1.0039 - val_acc: 0.6745


In [9]:
# Save model and weights
"""
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
"""
model_path = "drive/Colab Notebooks/GAN Medical Image/Models/keras_cifar10_trained_model_e2.h5"
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Saved trained model at drive/Colab Notebooks/GAN Medical Image/Models/keras_cifar10_trained_model_e2.h5 
10000/10000 [==============================] - 2s 171us/step
Test loss: 0.9940272159576417
Test accuracy: 0.6628


In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

gpg: keybox '/tmp/tmpl70aqqmu/pubring.gpg' created
gpg: /tmp/tmpl70aqqmu/trustdb.gpg: trustdb created
gpg: key AD5F235DF639B041: public key "Launchpad PPA for Alessandro Strada" imported
gpg: Total number processed: 1
gpg:               imported: 1
··········


In [2]:
!mkdir -p drive
!google-drive-ocamlfuse drive
!pip install -q keras

fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option


In [1]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 7496581354457912884, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 190447616
 locality {
   bus_id: 1
 }
 incarnation: 5054982191860526172
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"]